### Árvores de Decisão: Classificação e Regressão

As **árvores de decisão** são algoritmos de aprendizado supervisionado que podem ser usados tanto para **classificação** quanto para **regressão**. Elas funcionam de maneira hierárquica, dividindo os dados em subconjuntos com base em critérios que maximizam a separação entre as classes ou valores previstos. Cada divisão nos dados resulta em um nó na árvore, e os nós folhas representam as predições finais.

### Como Funciona uma Árvore de Decisão

O algoritmo cria uma árvore binária que divide os dados com base em perguntas de "sim ou não" (ou condições que podem ser numéricas ou categóricas). O objetivo é dividir os dados de forma que, ao final, cada subconjunto tenha uma maior homogeneidade possível, ou seja, que pertençam a uma mesma classe (para classificação) ou que os valores sejam o mais próximos possível (para regressão).

#### Classificação

No caso de **classificação**, o objetivo é separar os dados em grupos distintos com base nas classes disponíveis. Por exemplo, ao classificar se um e-mail é "spam" ou "não spam", a árvore de decisão usará características do e-mail (tamanho, palavras-chave, etc.) para fazer divisões sucessivas até chegar a uma classificação final.

#### Regressão

Já na **regressão**, o algoritmo tenta prever um valor numérico contínuo. Por exemplo, podemos prever o preço de uma casa com base em suas características (tamanho, localização, etc.). A árvore de decisão dividirá o conjunto de dados para minimizar a diferença entre o valor real e o valor previsto.

### Construção da Árvore de Decisão

O processo de construção da árvore envolve o seguinte:

1. **Seleção do Melhor Atributo**: Para cada nó, o algoritmo seleciona o melhor atributo para dividir os dados. O critério de escolha varia entre classificação e regressão.

2. **Cálculo da Impureza ou Erro**: 
    - Na **classificação**, o algoritmo calcula a impureza de um nó usando métricas como a **Entropia** ou o **Índice de Gini**.
    - Na **regressão**, ele tenta minimizar o **Erro Quadrático Médio (MSE)** ou outro erro associado à variância dos dados.

3. **Divisão Recursiva**: O processo é repetido recursivamente até que os dados estejam completamente separados ou até atingir um critério de parada, como uma profundidade máxima da árvore ou um número mínimo de amostras por nó.

4. **Poda**: Para evitar o **overfitting** (quando o modelo se ajusta demais aos dados de treino e perde capacidade de generalização), pode-se utilizar métodos de poda. A poda reduz o tamanho da árvore cortando nós que não agregam valor à predição.

### Métricas de Avaliação

#### Para Classificação:

- **Acurácia**: Proporção de predições corretas sobre o total de predições.
  
$$ \text{Acurácia} = \frac{\text{Número de predições corretas}}{\text{Número total de exemplos}} $$
  
- **Precisão**: Proporção de verdadeiros positivos sobre o total de predições positivas feitas pelo modelo.
  
$$ \text{Precisão} = \frac{\text{Verdadeiros Positivos}}{\text{Verdadeiros Positivos + Falsos Positivos}} $$
  
- **Revocação (Recall)**: Proporção de verdadeiros positivos em relação ao total de exemplos que realmente pertencem àquela classe.
  
$$ \text{Revocação} = \frac{\text{Verdadeiros Positivos}}{\text{Verdadeiros Positivos + Falsos Negativos}} $$
  
- **F1-Score**: Média harmônica entre a precisão e a revocação. É útil quando há um desbalanceamento nas classes.
  
$$ F1 = 2 \times \frac{\text{Precisão} \times \text{Revocação}}{\text{Precisão} + \text{Revocação}} $$

#### Para Regressão:

- **Erro Médio Absoluto (MAE)**: Média das diferenças absolutas entre os valores preditos e os valores reais.
  
$$ MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i| $$

- **Erro Quadrático Médio (MSE)**: Média das diferenças quadráticas entre os valores preditos e os valores reais.
  
$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

- **R-quadrado ($R^2$)**: Medida que indica o quão bem os valores preditos se ajustam aos valores reais.

$$ R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2} $$

### O Parâmetro `ccp_alpha` (Poda de Custo Complexo)

O parâmetro `ccp_alpha` é usado para controlar a **poda de custo complexo** em uma árvore de decisão. A poda é essencial para reduzir o overfitting em árvores muito profundas. O `ccp_alpha` permite especificar um valor de regularização que penaliza a complexidade da árvore, ou seja, árvores mais simples (com menos nós) são preferidas se o aumento de precisão for pequeno.

A fórmula usada para calcular a penalidade é dada por:

$$ R_\alpha(T) = R(T) + \alpha \cdot \left|T\right| $$

Onde:

- $R(T)$ é o erro da árvore $T$.
- $\alpha$ é o parâmetro de regularização (`ccp_alpha`).
- $\left|T\right|$ é o número de nós na árvore.

Quanto maior o valor de `ccp_alpha`, mais agressiva será a poda da árvore. Com um valor `ccp_alpha = 0`, a árvore será a mais complexa possível.

### Exemplo Prático em Python

Aqui está um exemplo de como usar árvores de decisão para classificação e regressão, e como aplicar a poda com o parâmetro `ccp_alpha`:


In [8]:
# Importar bibliotecas
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt

### Classificação

In [2]:
# Carregar dados do Titanic do openml
dados = fetch_openml(data_id=40945)
titanic = pd.DataFrame(dados.data)
titanic['survived'] = dados.target
titanic.sample(5)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,survived
759,3,"de Messemaeker, Mrs. Guillaume Joseph (Emma)",female,36.0,1,0,345572,17.4000,NaN,S,13,NaN,"Tampico, MT",1
1304,3,"Zabour, Miss. Hileni",female,14.5,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN,0
626,3,"Andersson, Miss. Ida Augusta Margareta",female,38.0,4,2,347091,7.7750,NaN,S,NaN,NaN,"Vadsbro, Sweden Ministee, MI",0
5,1,"Anderson, Mr. Harry",male,48.0,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY",1
617,3,"Ali, Mr. William",male,25.0,0,0,SOTON/O.Q. 3101312,7.0500,NaN,S,NaN,79.0,Argentina,0


In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   pclass     1309 non-null   int64   
 1   name       1309 non-null   object  
 2   sex        1309 non-null   category
 3   age        1046 non-null   float64 
 4   sibsp      1309 non-null   int64   
 5   parch      1309 non-null   int64   
 6   ticket     1309 non-null   object  
 7   fare       1308 non-null   float64 
 8   cabin      295 non-null    object  
 9   embarked   1307 non-null   category
 10  boat       486 non-null    object  
 11  body       121 non-null    float64 
 12  home.dest  745 non-null    object  
 13  survived   1309 non-null   category
dtypes: category(3), float64(3), int64(3), object(5)
memory usage: 116.8+ KB


## Dicionário de Dados
- **pclass**: Classe do bilhete
1 = 1ª Classe
2 = 2ª Classe
3 = 3ª Classe
- **name**: Nome do passageiro
- **sex**: Sexo do passageiro
- **age**: Idade do passageiro
- **sibsp**: Quantidade de cônjuges e irmãos a bordo
- **parch**: Quantidade de pais e filhos a bordo
- **ticket**: Número da passagem
- **fare**: Preço da Passagem
- **cabin**: Número da cabine do passageiro
- **embarked**: Porto no qual o passageiro embarcou
C = Cherbourg
Q = Queenstown
S = Southampton
- **boat**: bote salva-vidas
- **body**: Número de identificação do corpo
- **home.dest**: Destino final do passageiro
- **survived**: Informa se o passageiro sobreviveu ao desastre
0 = Não
1 = Sim

### Pré-Processamento
- Remover colunas irrelevantes
- Tratar valores faltantes
- Codificar variáveis categóricas
- Dividir os dados em treino e teste
- Treinar o modelo
- Avaliar o modelo
- Aplicar a poda
- Avaliar o modelo podado
- Realizar previsões
- Interpretar os resultados
- Apresentar conclusões
- Aplicar em casos de uso
- Considerações adicionais
- Referências

## Removendo colunas irrelevantes


In [4]:
titanic.drop(['name', 'ticket', 'cabin', 'boat', 'body', 'home.dest'], axis=1, inplace=True)
titanic.shape

(1309, 8)

- As colunas 'name', 'ticket', 'cabin' e 'home.dest' foram removidas por não serem relevantes para a análise.
- AS colunas 'boat' e 'body' foram removidas pois causas vazamento de informação, pois são informações que só são conhecidas após o acidente e indicam se o passageiro sobreviveu ou não.

In [6]:
# Deletar dados duplicados
titanic.drop_duplicates(inplace=True)
titanic.shape

(1114, 8)

## Verificando valores faltantes

In [7]:
titanic.isnull().sum()/len(titanic)*100

pclass       0.000000
sex          0.000000
age         12.567325
sibsp        0.000000
parch        0.000000
fare         0.089767
embarked     0.179533
survived     0.000000
dtype: float64

In [13]:
# Preencher valores faltantes na coluna 'age' utilizando KNN imputer
imputer = KNNImputer(n_neighbors=10)
titanic['age'] = imputer.fit_transform(titanic[['age']])


In [14]:
titanic.isnull().sum()/len(titanic)*100

pclass      0.000000
sex         0.000000
age         0.000000
sibsp       0.000000
parch       0.000000
fare        0.089767
embarked    0.179533
survived    0.000000
dtype: float64

In [15]:
titanic.dropna(inplace=True)
titanic.shape

## Transformando variáveis categóricas

In [30]:
# Função para contagem de valores únicos
def unique_values(df):
    for column in ['pclass', 'sex', 'sibsp', 'parch', 'embarked', 'survived']:
        print(f"{column}: {df[column].unique()}")


In [31]:
unique_values(titanic)

pclass: [1 2 3]
sex: ['female', 'male']
Categories (2, object): ['female', 'male']
sibsp: [0 1 2 3 4 5 8]
parch: [0 2 1 4 3 5 6 9]
embarked: ['S', 'C', 'Q']
Categories (3, object): ['C', 'Q', 'S']
survived: ['1', '0']
Categories (2, object): ['0', '1']


In [18]:

# transformar variáveis categóricas em numéricas
titanic = pd.get_dummies(titanic, columns=[], drop_first=True)

ValueError: could not convert string to float: 'female'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dados.data, dados.target, test_size=0.2, random_state=80)

# Criar o modelo
modelo_clf = DecisionTreeClassifier(random_state=80, criterion='gini')

# Treinar a árvore de decisão
modelo_clf.fit(X_train, y_train)

# Fazer predições
y_pred = modelo_clf.predict(X_test)

# Avaliar o modelo
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acuracia*100:.2f}")

In [ ]:
# Plotando a matriz de confusão
cf = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cf, display_labels=dados.target_names)
disp.plot()

In [ ]:
# Aplicando a poda, calculando os ccp_alpha
path = modelo_clf.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Lista para armazenar os resultados de validação cruzada
scores = []

# Para cada valor de ccp_alpha, treinamos o modelo
for alpha in ccp_alphas:
    modelo_clf = DecisionTreeClassifier(random_state=80, ccp_alpha=alpha).fit(X_train, y_train)
    scores.append(modelo_clf)

train_scores = [modelo_clf.score(X_train, y_train) for modelo_clf in scores]
test_scores = [modelo_clf.score(X_test, y_test) for modelo_clf in scores]

# Plotar os valores de ccp_alpha
fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("Acurácia")
ax.set_title("Acurácia x alpha do conjunto de dados de treino e teste")
ax.plot(ccp_alphas, train_scores, marker='o', label="treino",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="teste",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
# Mostrando os ccp_alpha
train_scores


In [ ]:
# Treinando com poda
modelo_podado = DecisionTreeClassifier(random_state=80, ccp_alpha=ccp_alphas[3])
modelo_podado.fit(X_train, y_train)

y_pred_podado = modelo_podado.predict(X_test)
acuracia_podada = accuracy_score(y_test, y_pred_podado)
print(f"Acurácia após poda: {acuracia_podada*100:.2f}")

#### Regressão

In [ ]:
# Carregar dados
dados = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(dados.data, dados.target, test_size=0.2, random_state=42)

# Criar o modelo
modelo = DecisionTreeRegressor(random_state=42)

# Treinar o modelo
modelo.fit(X_train, y_train)

# Fazer predições
y_pred = modelo.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
print(f"MSE: {mse:.2f}")

# Aplicando a poda
path = modelo.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Treinando com poda
modelo_podado = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alphas[-1])
modelo_podado.fit(X_train, y_train)

y_pred_podado = modelo_podado.predict(X_test)
mse_podado = mean_squared_error(y_test, y_pred_podado)
print(f"MSE após poda: {mse_podado:.2f}")

### Casos de Uso
#### Classificação:
- Diagnóstico Médico: Determinar se um paciente tem uma doença com base em sintomas e exames laboratoriais.
- Detecção de Fraude: Identificar se uma transação bancária é fraudulenta ou não.

#### Regressão:
- Previsão de Preço de Imóveis: Estimar o preço de uma casa com base em sua localização, tamanho e características.
- Previsão de Demanda: Prever a quantidade de produtos que será vendida em um determinado período.

### Conclusão
As árvores de decisão são ferramentas poderosas tanto para classificação quanto para regressão. Embora fáceis de interpretar, sua flexibilidade pode levar ao overfitting. O uso do parâmetro ccp_alpha é fundamental para aplicar poda e melhorar a generalização do modelo.


Para escolher o melhor valor de `ccp_alpha` (o parâmetro de poda de custo complexo) ao treinar uma árvore de decisão, o processo envolve equilibrar a complexidade da árvore e a precisão do modelo. Um valor adequado de `ccp_alpha` ajuda a controlar o **overfitting**, permitindo que a árvore generalize melhor para dados não vistos.

O processo básico para escolher o valor ideal de `ccp_alpha` envolve o uso de um **gráfico de validação cruzada** ou testes sucessivos com diferentes valores de `ccp_alpha`. A ideia é encontrar o valor que produz o melhor desempenho em dados de validação.

### 1. **Obtenha os valores de `ccp_alpha` disponíveis**:
Cada árvore de decisão tem um caminho de poda específico que pode ser representado por diferentes valores de `ccp_alpha`. O Scikit-learn oferece uma função chamada `cost_complexity_pruning_path` que retorna uma lista de valores de `ccp_alpha` e os erros correspondentes.


In [ ]:
path = modelo.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

A variável `ccp_alphas` conterá todos os possíveis valores de poda para a árvore, e impurities conterá o erro associado a cada valor de `ccp_alphas`.

### 2. Valide a árvore em diferentes valores de ccp_alpha:
Depois de obter os possíveis valores de ccp_alpha, você pode treinar múltiplos modelos, cada um com um valor de ccp_alpha, e avaliar o desempenho em um conjunto de validação.

Aqui está um exemplo prático de como isso pode ser feito:

In [ ]:
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Lista para armazenar os resultados de validação cruzada
scores = []

# Para cada valor de ccp_alpha, treinamos e validamos o modelo
for alpha in ccp_alphas:
    modelo = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
    score = cross_val_score(modelo, X_train, y_train, cv=5)
    scores.append(score.mean())

# Plotar os valores de ccp_alpha vs. desempenho (acurácia média)
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas, scores, marker='o', drawstyle="steps-post")
plt.xlabel('ccp_alpha')
plt.ylabel('Acurácia Média')
plt.title('Validação Cruzada para diferentes valores de ccp_alpha')
plt.show()


### 3. Escolha o ccp_alpha com melhor desempenho:
Após treinar a árvore para diferentes valores de ccp_alpha, você deve escolher aquele que maximiza o desempenho em termos de uma métrica de validação (como acurácia, F1-score, ou erro médio quadrático no caso de regressão). No gráfico gerado, o valor de ccp_alpha correspondente ao pico da curva geralmente é o ideal.

### 4. Testar o modelo com o melhor ccp_alpha:
Depois de identificar o melhor ccp_alpha, treine o modelo com esse valor e avalie seu desempenho final no conjunto de teste.

In [ ]:
# Escolha o melhor ccp_alpha (aquele que gerou a melhor acurácia)
melhor_ccp_alpha = ccp_alphas[scores.index(max(scores))]

# Treine a árvore final com o melhor ccp_alpha
modelo_final = DecisionTreeClassifier(random_state=42, ccp_alpha=melhor_ccp_alpha)
modelo_final.fit(X_train, y_train)

# Avalie no conjunto de teste
y_pred = modelo_final.predict(X_test)
acuracia_final = accuracy_score(y_test, y_pred)
print(f"Acurácia Final com melhor ccp_alpha: {acuracia_final:.2f}")


### 5. Validação Cruzada para maior robustez:
Além de apenas usar a divisão treino/teste, é recomendável usar validação cruzada para garantir que o valor escolhido de ccp_alpha seja realmente generalizável e não específico de um único conjunto de dados de validação.

In [ ]:
# Aplicando validação cruzada para diferentes valores de ccp_alpha
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Lista para armazenar os resultados de validação cruzada
scores = []

# Para cada valor de ccp_alpha, treinamos e validamos o modelo
for alpha in ccp_alphas:
    modelo = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
    score = cross_val_score(modelo, X_train, y_train, cv=5)
    scores.append(score.mean())
    
# Plotar os valores de ccp_alpha vs. desempenho (acurácia média)
plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas, scores, marker='o', drawstyle="steps-post")
plt.xlabel('ccp_alpha')
plt.ylabel('Acurácia Média')
plt.title('Validação Cruzada para diferentes valores de ccp_alpha')
plt.show()



Considerações Adicionais:
Overfitting vs. Underfitting: Um valor muito baixo de ccp_alpha resultará em uma árvore mais complexa (profunda), que pode sofrer de overfitting. Já um valor muito alto de ccp_alpha resultará em uma árvore muito simples, levando a underfitting (incapacidade de capturar padrões importantes nos dados).

Avaliando o Gráfico: No gráfico gerado, conforme o valor de ccp_alpha aumenta, a árvore se torna mais simples, o que pode levar a uma queda no desempenho. O objetivo é encontrar o ponto ótimo antes que a simplicidade da árvore comece a prejudicar a precisão.

Conclusão:
A escolha do melhor valor de ccp_alpha envolve um processo iterativo de validação e ajuste fino. Usar validação cruzada e um gráfico de desempenho vs. ccp_alpha é a melhor maneira de garantir que o modelo final seja suficientemente simples para generalizar bem e suficientemente complexo para capturar os padrões nos dados.